In [1]:
from chop.nn.quantized.functional import linearMXIntHardware
import torch


/home/luigirinaldi/anaconda3/envs/mxint-fork/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

width, exp_width, parallel = 8,3, [2,2]

quant_config = {
    "weight_width": width,
    "weight_exponent_width": exp_width,
    "weight_parallelism": parallel,
    "bias_width": width,
    "bias_exponent_width": exp_width,
    "bias_parallelism": parallel,
    "data_in_width": width,
    "data_in_exponent_width": exp_width,
    "data_in_parallelism": parallel
}

x = torch.rand([8,8])
weight = torch.rand([4,8])
bias = torch.rand([8,4])

layer = linearMXIntHardware(x,weight,bias=bias, config=quant_config)

print(layer)
layer.shape

tensor([[2.4727, 2.1992, 1.5781, 1.4648],
        [2.1562, 2.4805, 1.3750, 1.3477],
        [2.5664, 2.6055, 1.8984, 2.0977],
        [2.0508, 2.4023, 2.0391, 2.0938],
        [1.6562, 1.9414, 2.0195, 1.0586],
        [1.6562, 2.0820, 1.8633, 1.6055],
        [3.1250, 3.4961, 2.3203, 3.0859],
        [1.9805, 1.5156, 1.3516, 1.9336]])


/home/luigirinaldi/Documents/Imperial/Y4/ADL/coz-fork/src/chop/nn/quantizers/mxint_hardware.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reshaped_tensor = torch.tensor(reshaped_tensor.reshape(-1, p1 * p0))


torch.Size([8, 4])